# Crypto Homework 4

In [ ]:
import numpy as np
import hashlib as hlib

In [ ]:
#from provided_params import *
#from provided_sol import *

In [ ]:
from sciper_parameters import *

### Ex 1: Stop the counter! (ECDSA)

We use the `secp256k1` standard for the parameters of the curve. The curve is given by $y^2=x^3+ 7$

Provided parameters:
* `Q1_G`: point used as a generator for the EC?
* `Q1_p`: a prime int used to define the finite field we are working in `GF(p)`
* `Q1_pk`: pair of int used as the secretary's public key
* 
* `Q1a_arr`: 30 triplets containing: ("X voted for Y" as a msg, r, s)
* `Q1a_msg`: SOLUTION
* 
* `Q1b_arr`: 2 valid ballots that were published after the counter was stopped
* `Q1b_k`: An int to select a unique solution
* `Q1b_r`: SOLUTION
* `Q1b_s`: SOLUTION

#TODO: Delete the next lines for submission

* [Inspiration for SAGE](https://stackoverflow.com/questions/40434409/rdsa-implementation-on-sage)
* Ex 1b is the attack against Sony. The r's of 2 messages are the same ==> reuse of a nonce ==> retrieve the secret key: [implemented attack](https://github.com/Marsh61/ECDSA-Nonce-Reuse-Exploit-Example/blob/master/Attack-Main.py)
* Something else?

In [ ]:
# Provided code example
def msg2int(m):
    h_bytes = hlib.sha256(m.encode()).digest()
    h = int.from_bytes(h_bytes, "big")
    return h

In [ ]:
# Initialization
EC = EllipticCurve(GF(Q1_p), [0,7])
FF_n = FiniteField(EC.order())

#### Question a

Identify the frauders

In [ ]:
def verify_sign(EC, msg, r, s, G, Q, FF_n):
    # Build point G and Q
    G_EC = EC(G)
    Q_EC = EC(Q)
    
    # Hash the message and covert it to number
    h = msg2int(msg)

    # Compute w = 1/s mod p because we use it twice
    w = Integer(1/FF_n(s))
    
    # Compute verification
    verif = w*h*G_EC + w*r*Q_EC
    
    verif_x = verif.xy()[0]
    
    return r == verif_x

In [ ]:
# Check the signature of every messages
for msg, r, s in Q1a_arr:
    verif_ok = verify_sign(EC, msg, r, s, Q1_G, Q1_pk, FF_n)
    if not verif_ok:
        print(msg)

#### Question b

The main problem here is: How to find `d` from the secret value and the counter? I fact, when we have `d` the problem is utterly easy.

```
h0 = Hash(message_0)
h1 = Hash(message_1)
d = Private Key (unknown to attacker)
R  = r1 == r2
K  = K value that was used (unknown to attacker)
N  = integer order of G (part of public key)

From Signing Defintion
s0 = (h0 + d * R) / K Mod N
s1 = (h1 + d * R) / K Mod N

Rearrange 
K = (h0 + d * R) / s0 Mod N
K = (h1 + d * R) / s1 Mod N

Set Equal
(h0 + d * R) / s0 = (h1 + d * R) / s1     Mod N

s1*h0 + s1*d*R = s0*h1 + s0*d*R     Mod N

(s1*h0 - s0*h1) / (R * (s0-s1)) = d     Mod N

Solve for d (private key)
d Mod N = (s1 * h0 - s0 * h1) / (R * (s0 - s1))
d Mod N = (s1 * h0 - s0 * h1) * (R * (s0 - s1)) ** -1
```

In [ ]:
def compute_r(EC, k, G, FF_n):
    return FF_n((k * EC(G)).xy()[0])

In [ ]:
def extract_d(ballots, FF_n):
    assert(len(ballots) == 2)
    msg0, r0, s0 = ballots[0]
    msg1, r1, s1 = ballots[1]
    
    h0 = msg2int(msg0)
    h1 = msg2int(msg1)
    
    s0 = FF_n(s0)
    s1 = FF_n(s1)
    
    # Verify if the attack is feasible
    if r0 != r1:
        raise Exception('ERROR: the messages you provided are not sensible to this attack')
    
    d = (s1*h0 - s0*h1) / (r0 *(s0 - s1))
    
    return d

In [ ]:
def compute_s(h, d, r, k, FF_n):
    return FF_n((h + d*r)/k)

In [ ]:
r = compute_r(EC, Q1b_k, Q1_G, FF_n)

# Hash the message and build an integer from that
m = 'I won this election, by a lot!'
h = msg2int(m)

# Find d
d = extract_d(Q1b_arr, FF_n)

# Compute s
s = compute_s(h, d, r, Q1b_k, FF_n)

In [ ]:
print('r =', r)
print('s =', s)

---

### Ex 2

Provided parameters:
* `Q2_ct`: 18564 bytes
* `Q2_k`: composite int
* `Q2_n`: int the cardinality of the elliptic curve E, where Q2_n > Q2_p
* `Q2_p`: PRIME int characteristic of the field. Q2_p = Q2_n - 1
* `Q2_pt`: SOLUTION string message

The parameters `l` and `r1` are badly chosen!

The $j$-invariant is either 1728 or 0. It implies that the EC defined as $y^2 = x^3 + ax + b$ has either $a=0$ or $b=0$

The additive order of any point on the EC is Q2_p

I think that the trick for the ex2 is to find the order of P. 
The trick here is that the key has a point of symmetry. If P has order k then the value of x is the same for kP+i and kP-i.
So if this point of symmetry is in the padding, then you can decrypt the message because you get the key used for to encrypt M.
The order must divide the cardinality which has only "small" factors here.
Except 2 and 3, each factor of n is bigger than l.

The order of this elliptic curve is p+1 and is not a prime.


why symmetry？not period？
It's also true for the period, but we can't use it here since the prime factors of n are too big to see the period in the encryption

In [ ]:
import base64 as b64

In [ ]:
# Compute Legendre symbol of a/b
# kronecker(a,b)

#### SOME TESTS

In [ ]:
# Transform the ct into a bit sequence
ct_b64_decoded = b64.b64decode(Q2_ct[2:]).decode('utf-8')
ct_bit_seq = ''.join([format(ord(e), '08b') for e in ct_b64_decoded])
assert(len(ct_b64_decoded)*8 == len(ct_bit_seq))

#Search for palindromes using sage
w = Word(ct_bit_seq)
palindromes = sorted(w.palindromes(), key = lambda e:e.length(), reverse=True)

# Compute the location of the center of the longest palindrome
center_longest_pal = (w.find(palindromes[0]) + floor((palindromes[0].length()-1)/2), w.find(palindromes[0]) + floor((palindromes[0].length())/2))
print('Center longest palindrome:', center_longest_pal)

In [ ]:
nb_bits_before_center = center_longest_pal[0] + 1
nb_bits_after_center = w.length() - center_longest_pal[1]

half_len_pal = min(nb_bits_before_center, nb_bits_after_center)

# Split in two parts, left and right (before and after the center of the longest palindrome)
# WARNING: don't be stupid with the center of the palindrome. You should handle that
left = ct_bit_seq[center_longest_pal[0] + 1 - half_len_pal : center_longest_pal[0] + 1]
right = ct_bit_seq[center_longest_pal[1] : center_longest_pal[1] + half_len_pal]
assert(len(left) == len(right))
assert(len(left) == half_len_pal)

# XOR one part with the other one reversed
rev_left = [int(e) for e in reversed(left)]
right = [int(e) for e in right]


res_xor_bits = [rev_left[i] ^^ right[i] for i in range(len(rev_left))]

In [ ]:
# Bruteforce the padding to keep the message aligned
for bit_list in [res_xor_bits, list(reversed(res_xor_bits))]:
    for pad in range(8):
        bit_list_padded = [0]*pad + bit_list
        pt_byte = []
        for byte in range(floor(len(bit_list)/8)):
            pt_byte.append(''.join([str(i) for i in bit_list_padded[byte*8:(byte+1)*8]]))
            
        pt_msg = ''.join([chr(int(i,2)) for i in pt_byte])
        print(pt_msg)
        print('-'*60)

---
### Ex 3: MBCGA

Provided parameters:
* EX A: 3 rounds of encryption
* `Q3a_pt`: str  of 16 char ; plaintext
* `Q3a_k`: list of 128 int that is 0 or 1 ; The master key
* `Q3a_R`: list of 3 matrices of size 128x128 (3x128x128). The last elements are 0 or 1 ; The affine matrices.
* `Q3a_A`: 128x128 matrix of 0 or 1 (128x128) ; The key update matrix
* `Q3a_ct`: SOLUTION string of 128 bits (128x\[0,1\])
* 
* EX B: 2 rounds of decryption
* `Q3b_A`: 128x128 matrix of 0 or 1 (128x128) ; The key update matrix
* `Q3b_pt1`: str of 16 char ; plaintext 1
* `Q3b_ct1`: string of 128 bits (128x\[0,1\]) ; ciphertext of pt1
* `Q3b_R`: list of 2 matrices of size 128x128 (3x128x128). The last elements are 0 or 1 ; The affine matrices.
* `Q3b_ct2`: string of 128 bits (128x\[0,1\]) ; ciphertext of pt2 ==> we search pt2
* `Q3b_pt2`: SOLUTION str of 16 char ; plaintext 
* 
* EX C: 5 rounds of decryption
* `Q3b_A`: 128x128 matrix of 0 or 1 (128x128) ; The key update matrix
* `Q3b_pt1`: str of 16 char ; plaintext 1
* `Q3b_ct1`: string of 128 bits (128x\[0,1\]) ; ciphertext of pt1
* `Q3b_R`: list of 5 matrices of size 128x128 (3x128x128). The last elements are 0 or 1 ; The affine matrices.
* `Q3b_ct2`: string of 128 bits (128x\[0,1\]) ; ciphertext of pt2 ==> we search pt2
* `Q3b_pt2`: SOLUTION str of 16 char ; plaintext 



We are working in $Z2$

Multiplication done from the left (i.e. $K_2 = A\cdot K_1$)

Rounds are indexed from 1 (i.e. $k$ starts from 1)

We can import matrices with `Matrix(GF(2), L)` where `L` is a "matrix in a list type"

$S(000) = 000$

The Sbox is only applied to the first 3 bits (i.e. `state[:3]`) at each round.

#### Question a

We have 3 rounds in this exercices (i.e. `Q3a_R` contains 3 matrices)

In [ ]:
# Transform the message into a bit vector
pt_bin = ''.join([format(ord(i), '08b') for i in Q3a_pt])
pt_bin_vect = vector([int(i) for i in pt_bin])

In [ ]:
def sbox(bit_vect, F8):
    #build the function for the first 3 bits of the message. Check we get only 3 bits.
    assert(len(bit_vect) == 3)
    
    if(bit_vect == vector([0]*3)):
        return vector([0]*3)
    
    f_to_map = bit_vect[0] * x^2 + bit_vect[1] * x + bit_vect[2]
    to_inverse = F8(f_to_map)
    inv = to_inverse^(-1)
    
    res = inv.matrix().T[0][::-1]
    
    return vector([i for i in res])

In [ ]:
# Small test for the sbox function
F8.<x> = GF(2**3)
inv = sbox(vector([1,1,0]), F8) # Example: '110' = x^2 + x and its inverse is '011' = x + 1
inv

In [ ]:
def encrypt_round_k(msg_bit_vect, k, F8, affine_mats, key_update_mat, master_key_vect):
    #print('round', k)
    # SBox
    res_Sbox = sbox(msg_bit_vect[:3], F8)
    msg_Sboxed_vect = vector(list(res_Sbox) + list(msg_bit_vect[3:]))
    assert(len(msg_Sboxed_vect) == len(msg_bit_vect))
    assert(msg_Sboxed_vect[:3] == res_Sbox)
    assert(msg_Sboxed_vect[3:] == msg_bit_vect[3:])
    #print('after Sbox:', msg_Sboxed_vect)
    
    # Affine
    mat = Matrix(GF(2), affine_mats[k-1])
    res_affine = mat * msg_Sboxed_vect
    #print('after affine:', res_affine)
    
    #RoundKey (start k from 1)
    round_key = key_update_mat^k * master_key_vect
    
    #Compute final res
    size = len(round_key)
    final_res = [0]*size
    for i in range(size):
        final_res[i] = int(round_key[i]) ^^ int(res_affine[i])
    
    final_res_vect = vector(final_res)
    #print('after roundkey:', final_res_vect)
    
    return final_res_vect

In [ ]:
F8.<x> = GF(2**3)
master_key_vect = vector(Q3a_k)
key_update_mat = Matrix(GF(2), Q3a_A)

new_msg_vect = pt_bin_vect

# DO NOT FORGET to start k=1 (i.e. k is in {1,2,3} for the three rounds)
for round_nb in range(1,4):
    new_msg_vect = encrypt_round_k(msg_bit_vect=new_msg_vect,
                       k=round_nb,
                       F8=F8,
                       affine_mats=Q3a_R,
                       key_update_mat=key_update_mat,
                       master_key_vect=master_key_vect)
        
# Concat the vector to form a string
final_ct = ''.join([str(i) for i in new_msg_vect])
print(final_ct)

#### Question b

#TODO: Add explanation here

Observe that $S(S(x)) = x\implies S^{-1}(x) = S(x)$

#TODO: Delete the next lines below

Check online for "Elimination Gaussienne" (Telegram Turbo fiakZer)

In [ ]:
# Initialization
F8.<x> = GF(2**3)

pt1_bin = ''.join([format(ord(i), '08b') for i in Q3b_pt1])
pt1_bin_vect = vector([int(i) for i in pt1_bin])

ct1_vect = vector([int(i) for i in Q3b_ct1])
ct2_vect = vector([int(i) for i in Q3b_ct2])

L1 = Matrix(GF(2), Q3b_R[0])
L2 = Matrix(GF(2), Q3b_R[1])


# Start the decryption
pt1_affine = L1 * vector(list(sbox(pt1_bin_vect[:3], F8)) + list(pt1_bin_vect[3:]))

assert(len(pt1_affine) == len(pt1_bin_vect))

cts_xored = vector([ct1_vect[i] ^^ ct2_vect[i] for i in range(len(ct1_vect))])

reverse_cts_affine = L2.inverse() * cts_xored

for i in range(8):
    bruteforced_1 = [int(j) for j in list("{0:03b}".format(i))]
    reverse_cts_sbox = vector( bruteforced_1 + list(reverse_cts_affine[3:]))

    # By Vernam cipher
    pt2_affine = vector([Integer(reverse_cts_sbox[i]) ^^ Integer(pt1_affine[i]) for i in range(len(reverse_cts_sbox))])

    pt2_rdy_for_sbox = L1.inverse() * pt2_affine

    for j in range(8):
        bruteforced_2 = [int(l) for l in list("{0:03b}".format(j))]
        pt2 = vector(bruteforced_2 + list(pt2_rdy_for_sbox[3:]))


        # Convert pt2 binary vector into a message
        assert(len(pt2) == len(pt1_bin_vect))
        pt2_byte = []
        for byte in range(len(pt2) / 8):
            pt2_byte.append(''.join([str(i) for i in pt2[byte*8:(byte+1)*8]]))

        pt2_msg = ''.join([chr(int(i,2)) for i in pt2_byte])
        
        if pt2_msg.isalnum():
            print('try {:2}:'.format(i*8+j), pt2_msg)
    
#TODO: Delete next line before submission
#print(pt2_msg == Q3b_pt2)

#### Trying with majority function

In [ ]:
# Original S-box
for i in range(8):
    entry = vector([int(i) for i in list("{0:03b}".format(i))])
    print(entry, '=== SBOX ==>', sbox(entry, F8))

In [ ]:
# Do we need that ?! How can we use the majority function to ease this problem ?!?!?
# Here we build a sbox with the majority function and compare the output of the new S-box with the old one.

def s(a,b,c):
    ring = GF(2)
    f_majority = ring(a*b + a*c + b*c)
    s0 = ring(f_majority * (a+c+1) + a+b)
    s1 = ring(f_majority * (b+c+1) + a)
    s2 = ring(f_majority * (a+b+1) + a+b+c)
    return vector([s0,s1,s2])

for a in [0,1]:
    for b in [0,1]:
        for c in [0,1]:
            old_sbox = sbox(vector([a,b,c]),F8)
            new_sbox = s(a,b,c)
            print([a,b,c], '==>', old_sbox, new_sbox, old_sbox == new_sbox)

#### Question c

If you can't find a linear relationship for the Sbox conditioned on the majority of the input, try looking for an affine one



In [ ]:
# Initialization
F8.<x> = GF(2**3)

pt1_bin = ''.join([format(ord(i), '08b') for i in Q3c_pt1])
pt1_bin_vect = vector([int(i) for i in pt1_bin])

ct1_vect = vector([int(i) for i in Q3c_ct1])
ct2_vect = vector([int(i) for i in Q3c_ct2])

L1 = Matrix(GF(2), Q3c_R[0])
L2 = Matrix(GF(2), Q3c_R[1])
L3 = Matrix(GF(2), Q3c_R[2])
L4 = Matrix(GF(2), Q3c_R[3])
L5 = Matrix(GF(2), Q3c_R[4])

L1_inv = L1.inverse()
L2_inv = L2.inverse()
L3_inv = L3.inverse()
L4_inv = L4.inverse()
L5_inv = L5.inverse()


# Start the decryption
pt1_affine = L1 * vector(list(sbox(pt1_bin_vect[:3], F8)) + list(pt1_bin_vect[3:]))

assert(len(pt1_affine) == len(pt1_bin_vect))

cts_xored = vector([ct1_vect[i] ^^ ct2_vect[i] for i in range(len(ct1_vect))])

reverse_cts_affine = L5_inv * cts_xored

for s5 in range(8):
    bruteforced_1 = [int(j) for j in list("{0:03b}".format(s5))]
    reverse_cts_sbox_5 = vector( bruteforced_1 + list(reverse_cts_affine[3:]))

    pt2_rdy_for_sbox_4 = L4_inv * reverse_cts_sbox_5

    for s4 in range(8):
        bruteforced_2 = [int(j) for j in list("{0:03b}".format(s4))]
        reverse_cts_sbox_4 = vector( bruteforced_2 + list(pt2_rdy_for_sbox_4[3:]))

        pt2_rdy_for_sbox_3 = L3_inv * reverse_cts_sbox_4
        
        for s3 in range(8):
            bruteforced_3 = [int(j) for j in list("{0:03b}".format(s3))]
            reverse_cts_sbox_3 = vector( bruteforced_3 + list(pt2_rdy_for_sbox_3[3:]))

            pt2_rdy_for_sbox_2 = L2_inv * reverse_cts_sbox_3
            
            for s2 in range(8):
                bruteforced_2 = [int(j) for j in list("{0:03b}".format(s2))]
                reverse_cts_sbox_2 = vector( bruteforced_2 + list(pt2_rdy_for_sbox_2[3:]))
                
                # By Vernam cipher
                pt2_affine = vector([Integer(reverse_cts_sbox_2[i]) ^^ Integer(pt1_affine[i]) for i in range(len(reverse_cts_sbox_2))])

                pt2_rdy_for_sbox_1 = L1_inv * pt2_affine
                
                for s1 in range(8):
                    bruteforced_1 = [int(j) for j in list("{0:03b}".format(s1))]
                    pt2 = vector( bruteforced_1 + list(pt2_rdy_for_sbox_1[3:]))

                    
                    # Convert pt2 binary vector into a message
                    assert(len(pt2) == len(pt1_bin_vect))
                    pt2_byte = []
                    for byte in range(len(pt2) / 8):
                        pt2_byte.append(''.join([str(i) for i in pt2[byte*8:(byte+1)*8]]))

                    pt2_msg = ''.join([chr(int(i,2)) for i in pt2_byte])

                    if pt2_msg.isalnum():
                        print('try {:2}:'.format(s1+s2*8+s3*64+s4*512+s5*4096), pt2_msg)